# PyGSTi CHP Object Test

In [1]:
from __future__ import print_function #python 2 & 3 compatibility
import pygsti
pygsti.__version__

'0.9.9.2.post715+gcde6e292.d20210315'

In [2]:
from pygsti.objects.operation import CHPOp, StochasticCHPOp
from pygsti.tools.internalgates import standard_gatenames_chp_conversions

## CHPOp

In [3]:
c = CHPOp(['h 0', 'c 1 2'], 3)

In [4]:
print(c)
print(c.get_chp_str())
print(c.get_chp_str([3,4,5]))

CHP operation with labels: h 0, c 1 2

h 0
c 1 2

h 3
c 4 5



In [5]:
Gx = CHPOp('Gxpi 0', 1)
print(Gx)
print(Gx.get_chp_str())

CHP operation with labels: h 0, p 0, p 0, h 0

h 0
p 0
p 0
h 0



In [6]:
print(CHPOp('Gc20 0', 1))

CHP operation with labels: p 0, h 0, p 0, p 0, h 0



In [7]:
sc = StochasticCHPOp(['Gxpi 0', 'Gc20 0'], [0.2, 0.8], nqubits=1, seed_or_state=2021)
print(sc)
print(sc.get_chp_str([1]))
print(sc.get_chp_str([2]))
print(sc.get_chp_str([3]))
print(sc.get_chp_str([4]))

Stochastic CHP operation with probabilities and labels:
0.2: h 0, p 0, p 0, h 0
0.8: p 0, h 0, p 0, p 0, h 0

p 1
h 1
p 1
p 1
h 1

p 2
h 2
p 2
p 2
h 2

h 3
p 3
p 3
h 3

p 4
h 4
p 4
p 4
h 4



## CHPForwardSimulator + Explicit Model

In [8]:
chpexe = '/Users/sserita/Documents/notebooks/pyGSTi/2021-CHP/chp'
sim = pygsti.obj.CHPForwardSimulator(chpexe, shots=100)

In [9]:
#Initialize an empty Model object
model = pygsti.objects.ExplicitOpModel(['Q0', 'Q1'], simulator=sim, evotype='chp')

#Populate the Model object with states, effects, gates,
model['rho0'] = CHPOp([], nqubits=2)
model['Mdefault'] = pygsti.obj.ComputationalBasisPOVM(2, 'chp')

model['Gii'] = CHPOp([], nqubits=2)
model['Gxi'] = CHPOp(['Gxpi 0'], nqubits=2)
model['Gix'] = CHPOp(['Gxpi 1'], nqubits=2)
model['Gyi'] = CHPOp(['Gypi 0'], nqubits=2)
model['Giy'] = CHPOp(['Gypi 1'], nqubits=2)

print(model)

rho0 = CHP operation with labels: 


Mdefault = Computational(Z)-basis POVM on 2 qubits and filter None


Gii = 
CHP operation with labels: 


Gxi = 
CHP operation with labels: h 0, p 0, p 0, h 0


Gix = 
CHP operation with labels: h 1, p 1, p 1, h 1


Gyi = 
CHP operation with labels: p 0, p 0, h 0, p 0, p 0, h 0


Giy = 
CHP operation with labels: p 1, p 1, h 1, p 1, p 1, h 1






In [10]:
circ = pygsti.obj.Circuit(['Gix'])
model.probabilities(circ)

OutcomeLabelDict([(('00',), 0.0),
                  (('01',), 1.0),
                  (('10',), 0.0),
                  (('11',), 0.0)])

In [11]:
circ = pygsti.obj.Circuit(['Gix', 'Gxi'])
model.probabilities(circ)

OutcomeLabelDict([(('00',), 0.0),
                  (('01',), 0.0),
                  (('10',), 0.0),
                  (('11',), 1.0)])

In [12]:
circ = pygsti.obj.Circuit(['rho0', 'Gix', 'Gxi', 'Mdefault'])
model.probabilities(circ)

OutcomeLabelDict([(('00',), 0.0),
                  (('01',), 0.0),
                  (('10',), 0.0),
                  (('11',), 1.0)])

## ComposedOp + EmbeddedOp

In [13]:
from pygsti.objects.operation import ComposedOp, EmbeddedOp
from pygsti.objects.labeldicts import StateSpaceLabels

In [14]:
# ComposedOp
Gxx_composed = ComposedOp([CHPOp('Gxpi 0', nqubits=2), CHPOp('Gxpi 1', nqubits=2)])
print(Gxx_composed)
print(Gxx_composed.get_chp_str())

Composed operation of 2 factors:
Factor 0:
CHP operation with labels: h 0, p 0, p 0, h 0
Factor 1:
CHP operation with labels: h 1, p 1, p 1, h 1

h 0
p 0
p 0
h 0
h 1
p 1
p 1
h 1



In [15]:
# EmbeddedOp
Gxi_embedded = EmbeddedOp(['Q0', 'Q1'], ['Q0'], CHPOp('Gxpi 0', nqubits=1))
print(Gxi_embedded)
print(Gxi_embedded.get_chp_str())

Embedded operation with full dimension 4 and state space Q0(2)*Q1(2)
 that embeds the following 2-dimensional operation into acting on the ['Q0'] space
CHP operation with labels: h 0, p 0, p 0, h 0

h 0
p 0
p 0
h 0



In [16]:
Gix_embedded = EmbeddedOp(['Q0', 'Q1'], ['Q1'], CHPOp('Gxpi 0', nqubits=1))
print(Gix_embedded)
print(Gix_embedded.get_chp_str())

Embedded operation with full dimension 4 and state space Q0(2)*Q1(2)
 that embeds the following 2-dimensional operation into acting on the ['Q1'] space
CHP operation with labels: h 0, p 0, p 0, h 0

h 1
p 1
p 1
h 1



In [17]:
# Also see if I retain flexibility of targets
print(Gxi_embedded.get_chp_str([5,7]))
print(Gix_embedded.get_chp_str([5,7]))

h 5
p 5
p 5
h 5

h 7
p 7
p 7
h 7



In [18]:
# EmbeddedOp made of ComposedOps
Gxx_comp_embed = EmbeddedOp(['Q0', 'Q1', 'Q2', 'Q3'], ['Q1', 'Q2'], Gxx_composed)
print(Gxx_comp_embed)
print(Gxx_comp_embed.get_chp_str())
print(Gxx_comp_embed.get_chp_str([5, 6, 7, 8]))

Embedded operation with full dimension 16 and state space Q0(2)*Q1(2)*Q2(2)*Q3(2)
 that embeds the following 4-dimensional operation into acting on the ['Q1', 'Q2'] space
Composed operation of 2 factors:
Factor 0:
CHP operation with labels: h 0, p 0, p 0, h 0
Factor 1:
CHP operation with labels: h 1, p 1, p 1, h 1

h 1
p 1
p 1
h 1
h 2
p 2
p 2
h 2

h 6
p 6
p 6
h 6
h 7
p 7
p 7
h 7



## CHPForwardSimulator + LocalNoiseModel

In [19]:
# Step 1: Define stochastic Pauli noise operators
# Note that the probabilities here are the "error rates" that would be model parameters (currently just static)
noise_1q = StochasticCHPOp(['Gi 0', 'Gxpi 0', 'Gypi 0', 'Gzpi 0'], [0.97, 0.01, 0.01, 0.01], nqubits=1)

# Also need two-qubit version, which is currently just independent stochastic Pauli noise
noise_2q = ComposedOp([EmbeddedOp([0, 1], [0], noise_1q), EmbeddedOp([0, 1], [1], noise_1q)])

In [20]:
# Step 2: Define gate dict of noisy gates
# Using equivalent of XYICNOT modelpack
gatedict = {}
gatedict['Gi'] = noise_1q
gatedict['Gx'] = ComposedOp([CHPOp('Gxpi 0', nqubits=1), noise_1q])
gatedict['Gy'] = ComposedOp([CHPOp('Gypi 0', nqubits=1), noise_1q])
# Note that first Gcnot is now key in model, whereas second Gcnot is a standard gatename known to CHPOp constructor
gatedict['Gcnot'] = ComposedOp([CHPOp('Gcnot 0 1', nqubits=2), noise_2q])

In [21]:
from pygsti.objects.localnoisemodel import LocalNoiseModel
from pygsti.objects.spamvec import ComputationalSPAMVec

rho0 = CHPOp([], nqubits=4)
Mdefault = pygsti.obj.ComputationalBasisPOVM(4, 'chp')

chpexe = '/Users/sserita/Documents/notebooks/pyGSTi/2021-CHP/chp'
sim = pygsti.obj.CHPForwardSimulator(chpexe, shots=100)

ln_model = LocalNoiseModel(num_qubits=4, gatedict=gatedict, prep_layers=rho0, povm_layers=Mdefault,
                           availability={'Gcnot': [(0,1),(1,2),(2,3)]}, simulator=sim, evotype='chp')

In [22]:
# Step 4: Profit?? Worked way too quickly...
def print_implicit_model_blocks(mdl, showSPAM=False):
    if showSPAM:
        print('State prep building blocks (.prep_blks):')
        for blk_lbl,blk in mdl.prep_blks.items():
            print(" " + blk_lbl, ": ", ', '.join(map(str,blk.keys())))
        print()

        print('POVM building blocks (.povm_blks):')
        for blk_lbl,blk in mdl.povm_blks.items():
            print(" "  + blk_lbl, ": ", ', '.join(map(str,blk.keys())))
        print()
    
    print('Operation building blocks (.operation_blks):')
    for blk_lbl,blk in mdl.operation_blks.items():
        print(" " + blk_lbl, ": ", ', '.join(map(str,blk.keys())))
    print()

print_implicit_model_blocks(ln_model, showSPAM=True)

State prep building blocks (.prep_blks):
 layers :  rho0

POVM building blocks (.povm_blks):
 layers :  Mdefault

Operation building blocks (.operation_blks):
 layers :  Gi:0, Gi:1, Gi:2, Gi:3, Gx:0, Gx:1, Gx:2, Gx:3, Gy:0, Gy:1, Gy:2, Gy:3, Gcnot:0:1, Gcnot:1:2, Gcnot:2:3
 gates :  Gi, Gx, Gy, Gcnot



In [23]:
print(ln_model.operation_blks['gates']['Gx'])

Composed operation of 2 factors:
Factor 0:
CHP operation with labels: h 0, p 0, p 0, h 0
Factor 1:
Stochastic CHP operation with probabilities and labels:
0.97: 
0.01: h 0, p 0, p 0, h 0
0.01: p 0, p 0, h 0, p 0, p 0, h 0
0.01: p 0, p 0



In [24]:
print(ln_model.operation_blks['layers']['Gcnot', 1, 2])

Embedded operation with full dimension 16 and state space 0(2)*1(2)*2(2)*3(2)
 that embeds the following 4-dimensional operation into acting on the (1, 2) space
Composed operation of 2 factors:
Factor 0:
CHP operation with labels: c 0 1
Factor 1:
Composed operation of 2 factors:
Factor 0:
Embedded operation with full dimension 4 and state space 0(2)*1(2)
 that embeds the following 2-dimensional operation into acting on the [0] space
Stochastic CHP operation with probabilities and labels:
0.97: 
0.01: h 0, p 0, p 0, h 0
0.01: p 0, p 0, h 0, p 0, p 0, h 0
0.01: p 0, p 0
Factor 1:
Embedded operation with full dimension 4 and state space 0(2)*1(2)
 that embeds the following 2-dimensional operation into acting on the [1] space
Stochastic CHP operation with probabilities and labels:
0.97: 
0.01: h 0, p 0, p 0, h 0
0.01: p 0, p 0, h 0, p 0, p 0, h 0
0.01: p 0, p 0



In [25]:
# Step 5: Actually run circuits with local noise model
# TODO: Marginalized POVMs don't work yet, must specify num_lines as full space
circ = pygsti.obj.Circuit([('Gx', 1)], num_lines=4)
ln_model.probabilities(circ)

OutcomeLabelDict([(('0000',), 0.03),
                  (('0001',), 0.0),
                  (('0010',), 0.0),
                  (('0011',), 0.0),
                  (('0100',), 0.97),
                  (('0101',), 0.0),
                  (('0110',), 0.0),
                  (('0111',), 0.0),
                  (('1000',), 0.0),
                  (('1001',), 0.0),
                  (('1010',), 0.0),
                  (('1011',), 0.0),
                  (('1100',), 0.0),
                  (('1101',), 0.0),
                  (('1110',), 0.0),
                  (('1111',), 0.0)])

In [26]:
circ = pygsti.obj.Circuit([('Gx', 1), ('Gcnot', 1, 2)], num_lines=4)
ln_model.probabilities(circ)

OutcomeLabelDict([(('0000',), 0.04),
                  (('0001',), 0.0),
                  (('0010',), 0.03),
                  (('0011',), 0.0),
                  (('0100',), 0.04),
                  (('0101',), 0.0),
                  (('0110',), 0.89),
                  (('0111',), 0.0),
                  (('1000',), 0.0),
                  (('1001',), 0.0),
                  (('1010',), 0.0),
                  (('1011',), 0.0),
                  (('1100',), 0.0),
                  (('1101',), 0.0),
                  (('1110',), 0.0),
                  (('1111',), 0.0)])

In [27]:
# Could also define correlated noise for 2-qubit error?
noise_2q_correlated = StochasticCHPOp([
    ['Gi 0', 'Gi 1'],       # II
    ['Gxpi 0', 'Gi 1'],     # XI
    ['Gi 0', 'Gxpi 1'],     # IX
    ['Gxpi 0', 'Gxpi 1'],   # XX (Will make 10x more prevalent)
    ['Gypi 0', 'Gypi 1'],   # YY
    ['Gzpi 0', 'Gzpi 1']],  # ZZ
    [0.86, 0.01, 0.01, 0.1, 0.01, 0.01], nqubits=2)

gatedict = {}
gatedict['Gi'] = CHPOp('Gi 0', nqubits=1)
gatedict['Gx'] = CHPOp('Gxpi 0', nqubits=1)
gatedict['Gy'] = CHPOp('Gypi 0', nqubits=1)
# Note that first Gcnot is now key in model, whereas second Gcnot is a standard gatename known to CHPOp constructor
gatedict['Gcnot'] = ComposedOp([CHPOp('Gcnot 0 1', nqubits=2), noise_2q_correlated])

In [28]:
rho0 = CHPOp([], nqubits=4)
Mdefault = pygsti.obj.ComputationalBasisPOVM(4, 'chp')

chpexe = '/Users/sserita/Documents/notebooks/pyGSTi/2021-CHP/chp'
sim = pygsti.obj.CHPForwardSimulator(chpexe, shots=100)

ln_model_corr = LocalNoiseModel(num_qubits=4, gatedict=gatedict, prep_layers=rho0, povm_layers=Mdefault,
                           availability={'Gcnot': [(0,1),(1,2),(2,3)]}, simulator=sim, evotype='chp')

In [29]:
circ = pygsti.obj.Circuit([('Gx', 1), ('Gcnot', 1, 2)], num_lines=4)
ln_model_corr.probabilities(circ)

OutcomeLabelDict([(('0000',), 0.09),
                  (('0001',), 0.0),
                  (('0010',), 0.01),
                  (('0011',), 0.0),
                  (('0100',), 0.01),
                  (('0101',), 0.0),
                  (('0110',), 0.89),
                  (('0111',), 0.0),
                  (('1000',), 0.0),
                  (('1001',), 0.0),
                  (('1010',), 0.0),
                  (('1011',), 0.0),
                  (('1100',), 0.0),
                  (('1101',), 0.0),
                  (('1110',), 0.0),
                  (('1111',), 0.0)])

In [45]:
circ = pygsti.obj.Circuit([('Gx', 1), ('Gcnot', 1, 2)])
ln_model_corr.probabilities(circ)

ValueError: substring not found